<img align='center' style='max-width: 1100px' src='banner.jpg'>

<img align="right" style="max-width: 250px; height: auto" src="sfao_logo.png">

## Agentic Auditing - Accounting Journal Entry Testing

Lab 01 - 63rd World Continuous Auditing & Reporting Symposium, Newark, NJ 

Our lab environment is based on [**Jupyter Notebooks**](https://jupyter.org/), which provide an interactive platform for performing a variety of statistical evaluations and data analyses. In this lab, we will explore how to apply **Artificial Intelligence (AI)** to implement and perform audit tasks in an agentic setting. 

The objective is to leverage multiple AI agents that can collaboratively analyze accounting journal entries, simulating the work of human auditors. This approach is based on the concept of [**Agentic Auditing**](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4909147), where specialized AI agents are assigned distinct tasks, such as anomaly detection, compliance verification, and approval flow analysis. Each agent operates autonomously but works as part of a team, ensuring that audit processes are thorough and efficient. The figure below illustrates the **agentic journal entry testing process** we aim to establish in this lab.

<img align="center" style="max-width: 1000px" src="./audit_process.png">

**Figure 1: Artificial Intelligence Agents**

We will implement and run multiple agents using the [**CrewAI**](https://www.crewai.com/) (<img align="center" style="max-width: 60px" src="./crewai_logo.png">) framework to collaboratively audit journal entries. `CrewAI` is an open source framework hat enables developers to build and deploy automated workflows using multiple AI agents that collaborate to perform complex tasks. The corresponding `CrewAI` GitHub repository can be found via the following link: [https://github.com/crewAIInc/crewAI](https://github.com/crewAIInc/crewAI)

Using the `CrewAI` framework, we will implement and deploy a team of AI agents (as illustrated in Fig. 1) to audit accounting journal entries according to the [International Standard on Auditing (ISA) 240](https://www.ifac.org/_flysystem/azure-private/publications/files/A012%202013%20IAASB%20Handbook%20ISA%20240.pdf). Each audit agent will perform specific tasks such as (i) document analysis, (ii) database queries, and (iii) report writing. After setting up the agents, we will have them work together in a sequental process to the task of perform journal entry testing.

As always, pls. don't hesitate to ask all your questions either during the lab, post them in our CANVAS (StudyNet) forum (https://learning.unisg.ch), or send us an email (using the course email).

## 1. Lab Objectives:

After today's lab, you should be able to:

> 1. **Understand Agentic Auditing:** Grasp the basics of using AI agents for auditing tasks.
> 2. **Implement AI Agents with crewAI:** Set up and configure agents to perform audit tasks.
> 3. **Apply Agents for Journal Entry Testing:** Use agents to detect anomalies in journal entries.
> 4. **Evaluate Agent Performance:** Analyze the performance of agents using audit-relevant metrics.
> 5. **Report Audit Outcomes:** Visualize and summarize audit results.

By the end of the lab, you will understand how to set up AI agents using the `CrewAI` framework, assign them specific audit tasks, and have them work together to perform the task of Journal Entry Testing.

## 2. Setup of the Jupyter Notebook Environment

We need to import several Python libraries that facilitate data analysis and visualization. We will primarily use `crewAI`, `SqlAlchemy`, `Langchain`, and a few utility libraries throughout this lab:

In [ ]:
# import text processing, database and utility libraries
import os
import requests
import pandas as pd
import tabulate as tb
from datetime import datetime
from IPython.display import Markdown, display
from textwrap import dedent
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Date, insert

Install the `crewAI` agentic framework:

In [ ]:
# install the crewai library and tools
# ! pip install crewai
# ! pip install crewai_tools

Import the `crewAI`agentic framework library:

In [ ]:
# import the crewai library and tools
from crewai import Crew, Agent, Process, Task
from crewai_tools import BaseTool

Install the `langchain` LLM chaining library:

In [ ]:
# ! pip install langchain_openai 

Import the `langchain` LLM chaining library:

In [ ]:
# import langchain
from langchain_openai import ChatOpenAI

## 3. Setting up the Large Language Model (LLM)

In this section, we will set up two different Large Language Models (LLMs) that our agents will use to perform audit-related tasks. We start with a locally hosted LLM using the [**Ollama**](https://ollama.com/) application and then explore accessing the remotely hosted [**ChatGPT**](https://platform.openai.com/docs/overview) via OpenAI’s API. This setup allows us to experiment with using both local and cloud-based LLMs in our agentic auditing tasks. In both settings we will use the **Langchain** `ChatOpenAI` interface to access the models.

### 3.1 Setting up a Local LLM with Ollama

The **Ollama** application provides the ability for running LLMs locally. We will setup and using the **LLaMA3** model published by Meta.ai through **Ollama**. This will enable us with local execution of the model, reducing latency and enhancing data privacy. To achieve this let's first download via the following link for your OS to download and install the application: [https://ollama.com/download](https://ollama.com/download).

After successful installation, let's run the `ollama list` command from your terminal to review the available models:

In [ ]:
! ollama list

Next, we aim to utilize a **LLaMA 3.2.** model that is instruction-tuned and of size 3 billion parameters(text in/text out). The model is a text only model familiy which was optimized for multilingual dialogue use cases, including agentic retrieval and summarization tasks. Let's download the **LLaMA 3.2** model within the **Ollama** application using the `ollama pull` terminal command:

In [ ]:
! ollama pull llama3.2:3b

Next, we briefly double check of the model was successfully downloaded:

In [ ]:
! ollama list

Looks great. Let's now run the model locally on your device using the `ollama run` command and the corresponding model name:

In [ ]:
! ollama run llama3.2:3b

Now that the model is running, we will use the **Langchain** `ChatOpenAI` interface to connect to a to a locally hosted **LLaMA 3.2** model:

In [ ]:
# initialize the local LLaMA 3.2 by meta.ai
llm_llama = ChatOpenAI(model='llama3.2:3b', base_url='http://localhost:11434/v1', api_key='Na')

Next, let’s have some fun testing the local **LLaMA3** model. We’ll simulate a scenario where the AI is asked to audit an expense report for a quirky employee who might be trying to sneak in some personal purchases. This test will ensure our model is responding correctly:

In [ ]:
# define a fun audit prompt 
prompt = """
Imagine you're an auditor reviewing an expense report from an employee who claims all expenses are legitimate business costs. 
Here's a list of items they've submitted:

1. Office supplies - $100
2. Coffee machine - $250
3. Unicorn onesie - $45
4. 10-pound gummy bear - $30
5. "Business trip" to Disneyland - $2000
6. Sushi for team meeting - $150

Please identify which items are legitimate business expenses and which might require further investigation, with a bit of humor in your response. 
"""

Next, the local **LLaMA3** model is tasked with determining which items in the expense report are legitimate and which require further investigation. The model should provide a humorous yet practical analysis of the expenses:

In [ ]:
# test the model with this fun audit prompt
print(llm_llama.invoke(prompt).content)

### 3.2 Setting up a cloud-based LLM with Langchain

The **OpenAI API** allows us to use language models like **GPT-4** (ChatGPT) over the cloud. In this section, we’ll set up ChatGPT by configuring an API key and establishing a connection to OpenAI’s servers. This approach is beneficial for users who want access to a high-performance model but do not require local hosting.

To get started, create an OpenAI account and obtain an API key from OpenAI’s **API dashboard** available via: [https://platform.openai.com](https://platform.openai.com)

After obtaining the API key, let’s set it as an environment variable. This secures the key and allows our code to interact with the **GPT-4** model:

In [ ]:
# set the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = 'your_actual_openai_api_key_here'  # Replace with your actual OpenAI API key

Now, we’ll use the `ChatOpenAI` library to initialize the **GPT-4** model using the OpenAI API. Make sure the API key is correctly set before running this code.

In [ ]:
# initialize ChatGPT with OpenAI's GPT-4 model
llm_gpt4 = ChatOpenAI(model='gpt-4', api_key=os.getenv('OPENAI_API_KEY'))

With **GPT-4** successfully connected via OpenAI’s API, we can now use this cloud-based LLM to assist in our agentic auditing tasks. This setup gives us access to powerful language capabilities while leveraging OpenAI’s infrastructure.

Next, let’s have some fun testing the cloud-based **GPT-4** model! Instead of the usual audit tasks, we’ll ask it a philosophical question about the meaning of life. This playful prompt will let us see how well the model can handle requests showcasing its ability to think analytically, just as any good auditor would:

In [ ]:
# define a fun audit prompt
prompt = """
Imagine you're an auditor known for precision and rigor, and one day, a curious question lands on your desk: 

"What is the meaning of life?" 

Using your audit skills, approach this question as if you’re conducting an audit. What evidence might you seek, 
what factors would you evaluate, and what red flags could arise in your quest to give a well-substantiated answer?
Add a touch of humor in your response, because, after all, who better to weigh in on life’s mysteries than an auditor?
"""

Next, let's send the prompt the cloud-based **GPT-4** model. The model should provide a humorous response to the query:

In [ ]:
# test the model with this fun audit prompt
print(llm_gpt4.invoke(prompt).content)

In this section, we explored two powerful ways to integrate language models into our auditing toolkit: setting up a local model with **Ollama** and accessing the **GPT-4** model via OpenAI’s cloud-based service. These tools give us flexibility in performing agentic audit tasks, balancing privacy with capability. Now that our environment is set up, we’re ready to dive into more hands-on applications, where these models will be utilized as audit agents within a given auditing process.

## 4. Establishing the Agentic Journal Entry Testing

In this section, we will guide you through creating the foundational agents of our audit crew. We start by establishing a **Senior Auditor** agent, who can analyze complex organizational guidelines and summarize them for audit purposes. Then, we will expand to include additional agents, forming a collaborative team to carry out journal entry testing in an agentic workflow.

### 4.1 Establishing a Single Auditor Agent

The **Senior Auditor** is the first agent in our audit team, designed to analyze complex guidelines and generate actionable insights. Its primary task is to read organizational guidelines and summarize them concisely for auditing purposes. This agent’s "brain" is powered by the **LLaMA 3.2 model** set up earlier, which allows it to handle language-based tasks like analyzing and summarizing text (see Figure 2). Let's walk through the steps to create and assign tasks to this agent.


<img align='center' style="max-width: 850px" src='./senior_auditor_agent.png'>

**Figure 2: Senior Auditor Agent - Tools and Capabilities**

The **Senior Auditor** agent will need a tool to read the organizational guidelines.  We provide this capability by defining the `ReadGuidelines` tool, which will read a text file containing the guidelines and return its content. This will allow the agent to access and process the guidelines.

In [ ]:
# define the ReadGuidelines tool for the Senior Auditor agent
class ReadGuidelines(BaseTool):
    
    # set tool attributes
    name: str = 'Read Guidelines'  # name of the tool
    description: str = 'Reads organizational guidelines and returns the guideline content.'  # tool description
    result_as_answer: bool = True  # ensures the output is given as an answer

    # this method reads a text file containing the guidelines
    def _run(self, query: str) -> str:

        try:
            # define the remote file path
            file_path = 'https://raw.githubusercontent.com/GitiHubi/AgenticAuditing/master/guideline.txt'
            
            # open and read the content of the guideline file
            content = requests.get(file_path).text
            
            # return the file content as a string
            return str(content)
        
        except Exception as exception:
            
            # return error message if file reading fails
            return str(exception)

The **ReadGuidelines** tool enables the **Senior Auditor** agent to access and read the organization's guideline document, supporting the agent in summarizing key points for audit purposes. Let's instantiate the tool:

In [ ]:
# instantiate the ReadGuidelines tool
rga = ReadGuidelines()

With the `ReadGuidelines` tool set up, we now define the **Senior Auditor** agent. This agent will read organizational guidelines and summarize key information for auditing. The agent’s "brain" leverages the **LLaMA 3.2 model**, providing it with advanced language comprehension and summarization capabilities.

The **Senior Auditor** agent has the following attributes:
- **Role:** Senior Auditor.
- **Goal:** To extract and summarize key information from organizational guidelines.
- **Backstory:** Known for simplifying complex guidelines into actionable insights.

In [ ]:
# define the Senior Auditor agent, using LLaMA 3.2 as its language model
senior_auditor = Agent(
    role='Senior Auditor',  # the agent's role
    goal="Analyze comprehensive organizational guidelines and extract their main points.",  # the agent's task focus
    
    # detailed backstory providing context for the agent's role
    backstory="""You are a skilled Senior Auditor, known for your expertise in analyzing complex guidelines 
    and translating them into concise, actionable insights. Your summaries are critical for guiding 
    other audit team members in compliance and audit assessments.""",
    
    verbose=True,  # enables detailed logging for transparency during execution
    allow_delegation=False,  # disables task delegation for this agent to maintain focused task handling
    llm=llm_gpt4,  # specifies the LLaMA 3.2 model as the agent's "brain" for language-based tasks
    tools=[rga]  # attaches the ReadGuidelines tool to enable reading and processing guideline documents
)

The **Senior Auditor** agent will be tasked with analyzing and summarizing the organization’s guidelines. The goal is to produce a concise, accessible summary, especially focused on filtering criteria relevant to auditing payments.

In summary, the task should comply with the following criteria:
- **Criteria 1:** Be clear and concise, using minimal jargon.
- **Criteria 2:** Focus on filtering criteria relevant to auditing payments.
- **Criteria 3:** Explicitly state the filtering criteria that the IT auditor should use.
- **Criteria 4:** Provide the summary in just two or three sentences.

In [ ]:
# define the task for the Senior Auditor agent
analyze_guideline = Task(
    description=dedent("""\
        Summarize the given guideline and extract its main gist in a very concise and short manner for an IT auditor.

        Utilize the provided guideline text and insights to complete your task effectively.

        Your summary should be:
        1. Clear and concise, avoiding complex jargon to ensure readability.
        2. Focused on the key points relevant to deriving a SQL query for auditing payments.
        3. Explicitly state the filtering criteria that the IT auditor should use when analyzing the payments to identify potential violations of the guideline.
        4. Short, encompassing a total of two to three sentences (this is very important).
    """),
    expected_output=dedent("""\
        A concise and clear summary of the guideline, tailored for an IT auditor, including explicit filtering criteria for analyzing payments.
    """),
    agent=senior_auditor,  # assign the task to the senior auditor agent
    max_attempts=1,        # set the number of attempts allowed for this task
    output_file='task_01.txt'  # file to store the task output
)

With the task now defined, we can execute it. The **Senior Auditor** agent will process the guidelines and produce a summarized output according to the specified criteria.

In [ ]:
analyze_guideline.execute_sync();

### 4.2 Establishing the Team of Agents for Journal Entry Testing

In this section, we expand the audit team by introducing two additional agents: the **Senior IT Auditor** and the **Audit Manager**. Each of these agents brings specialized capabilities to the audit process, enabling a comprehensive workflow for journal entry testing. The **Senior IT Auditor** will focus on analyzing payment records within the database, ensuring compliance with organizational guidelines. Meanwhile, the **Audit Manager** will compile findings into a structured report, making them accessible to a non-technical audience.

#### 4.2.1 Establishing the Senior IT Auditor Agent

The second **Senior IT Auditor** agent specializes in querying databases to detect potential guideline violations. This agent’s \"brain\" is also powered by the **LLaMA 3.2 model**, enabling it to perform language-based tasks related to query formulation and data analysis (see Figure 3). Its primary task will be to translate audit guidelines into **SQL queries**, analyze journal entries, and identify high-value transactions that could indicate compliance issues. 

Let's walk through the steps to create and assign tasks to this agent.

<img align='center' style="max-width: 850px" src='./senior_it_auditor_agent.png'>

**Figure 3: Senior IT Auditor Agent - Tools and Capabilities**

Before we define the **Senior IT Auditor** agent, we need to set up an in-memory SQLite database that contains sample payment records. This will allow the agent to run SQL queries on these records to check for potential violations of the organizational guidelines.

In [ ]:
# define the metadata object
metadata_obj = MetaData()

# Define the schema for a payments table
payments = Table(
    "payments",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("text", String(4), nullable=False),
    Column("amount", Float, nullable=False),
    Column("date", Date, nullable=False),
)

# insert sample data for payments
payment_data = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 1029, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 1225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 1798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

# create the in-memory SQLite engine and apply the schema
engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

# insert data into the payments table
with engine.begin() as conn:
    for payment in payment_data:
        stmt = insert(payments).values(
            id=payment[0],
            text=payment[1],
            amount=payment[2],
            date=payment[3]
        )
        conn.execute(stmt)

Now that we have set up the in-memory payments database, we need to define a function that allows the **Senior IT Auditor** to run SQL queries on the database. This function will execute the queries and return the results in a structured format.

In [ ]:
# define a function to execute a SQL query on the database
def run_query(query):
    try:
        # execute the query and fetch results
        result = pd.read_sql(query, con=engine)
        
        if result.empty:
            return "No results found."
        
        # format and return the result as a table
        table = tb.tabulate(result, headers=result.columns, tablefmt="grid")
        
        return table
    
    except Exception as e:
        return str(e)

The **Senior IT Auditor** will use the `QueryDatabase` tool to run SQL queries on the payment records. This tool takes a query as input, executes it on the database, and returns the results.

In [ ]:
# define the QueryDatabase tool
class QueryDatabase(BaseTool):

    name: str = "Query database"
    description: str = "Execute SQL database query and return the query result."
    result_as_answer: bool = True

    # define database query
    def _run(self, query: str) -> str:

        try:
            query = query.strip().replace('"', '')

            # echoing cleaned query
            print(f"\nExecuting cleaned query: {query}")

            if not query:
                return "No SQL query provided."

            result = run_query(query)

            return result

        except Exception as exception:
            return str(exception)

The **QueryDatabase** tool allows the **Senior IT Auditor** agent to execute SQL queries on the payments database, facilitating efficient auditing of high-value payments based on the specified filtering criteria. Let's instantiate the tool:

In [ ]:
# instantiate the QueryDatabase tool
qdb = QueryDatabase()

Next, we define the **Senior IT Auditor** agent. This agent is responsible for converting the organizational guideline summary into SQL queries to determine potential guideline violations in the payments database. The agent will use the `QueryDatabase` tool to execute these queries.

The **Senior IT Auditor** agent has the following attributes:
- **Role:** Senior Auditor.
- **Goal:** Convert guideline summaries into SQL queries for auditing payment records.
- **Backstory:** A highly skilled IT auditor known for analyzing complex systems and ensuring compliance.

In [ ]:
# define the Senior IT Auditor agent with a focus on querying payment records for potential compliance issues
senior_it_auditor = Agent(
    role='Senior IT Auditor',  # role designation for the agent
    goal=dedent("""\
        convert an organizational guidelines summary into a SQL query to determine guideline violations in the database of payment records.
    """),  # goal statement to translate audit requirements into actionable queries
    
    # detailed backstory highlighting the agent's role and expertise
    backstory=dedent("""\
        you are a highly skilled IT auditor at a leading financial auditing firm. 
        your expertise includes identifying system vulnerabilities, ensuring regulatory compliance, and optimizing payment processes.
        you excel in analyzing complex IT systems and providing actionable recommendations to enhance security and efficiency.
    """),
    
    verbose=True,  # enable detailed logging to monitor task execution
    allow_delegation=False,  # prevents task delegation to ensure focused task handling by this agent
    llm=llm_gpt4,  # the agent's "brain," utilizing the LLaMA3 model for handling language-based tasks
    tools=[qdb]  # attach the QueryDatabase tool for SQL query execution on payment records
)

The **Senior IT Auditor** agent will be tasked with translating the filtering criteria provided by the **Senior Auditor** into a SQL query, which will be executed on the payment records database to identify any potential guideline violations, especially high-value transactions.

In summary, the task should comply with the following criteria:
- **Criteria 1:** Derive a clear SQL query based on the Senior Auditor’s filtering criteria.
- **Criteria 2:** Use `*` to retrieve all fields in the query result.
- **Criteria 3:** Return query results in a structured, readable format.

In [ ]:
# define the Query Database task for the Senior IT Auditor agent
query_database = Task(
    description=dedent("""\
        Translate the filtering criteria provided by the **Senior Auditor** into a SQL query and execute it to audit the relevant database for payments exceeding a specific threshold.
        
        Your actions should include:
        
        1. Use the explicit filtering criteria from the **Senior Auditor** to formulate a SQL query.
        2. Ensure that the query retrieves all relevant fields by using `*` instead of specifying individual field names.
        3. Execute the query and format the results in a structured format, such as JSON or a well-formatted table.
        
        The input should always adhere to this format:
        
        {"query": "SELECT * FROM payments WHERE <filtering_criteria>"}

    """),
    expected_output=dedent("""\
        A table containing all fields (id, text, amount, date) for payments that meet the filtering criteria provided by the Senior Auditor.
    """),  # expected output in a clear and structured format
    
    agent=senior_it_auditor,  # assign the task to the Senior IT Auditor agent
    max_attempts=1,           # limit task attempts to one to maintain workflow control
    output_file='task_02.txt' # specify output file to store the task results
)

#### 4.2.2 Establishing the Audit Manager Agent

The third **Audit Manager** agent synthesizes the findings of the audit team into a clear, accessible report. This agent is skilled in distilling complex audit data into concise summaries, helping stakeholders understand key insights without technical jargon. The **Audit Manager** doesn’t require external tools; it leverages the **LLaMA 3.2 model** to generate structured and actionable reports (see Figure 4).

Let's walk through the steps to create and assign tasks to this agent.

<img align='center' style='max-width: 900px' src='./audit_manager_agent.png'>

**Figure 4: Senior Audit Manager Agent - Tools and Capabilities**

We now define the **Audit Manager** agent. This agent is tasked with synthesizing findings into a clear and comprehensive audit report that communicates the audit process, findings, and actionable recommendations effectively. Leveraging the insights provided by the **Senior IT Auditor**, the **Audit Manager** delivers impactful reports designed for easy interpretation by diverse stakeholders.

The **Audit Manager** agent has the following attributes:
- **Role:** Audit Manager.
- **Goal:** To compile audit findings into an insightful, accessible, and actionable report.
- **Backstory:** A celebrated auditor for transforming complex audit data into clear insights.


In [ ]:
audit_manager = Agent(
    role='Manager Auditor',
    goal=dedent("""\
        Create comprehensive and compelling audit reports that provide clear insights and actionable recommendations.
    """),
    backstory=dedent("""\
        You are a renowned Audit Manager, celebrated for your ability to produce insightful and impactful audit reports.
        Your reports distill complex audit findings into clear, accessible, and actionable insights, making them valuable to stakeholders.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm_gpt4, # the agent will use the LLaMA3 model
    tools=[]  # the Audit Manager doesn't need any tools here
)

The **Audit Manager** agent will be tasked with compiling the audit findings from the **Senior IT Auditor** into a comprehensive report. The report should provide stakeholders with a structured overview of the audit process, insights into the findings, and actionable recommendations based on the audit results.

In summary, the task should comply with the following criteria:
- **Criteria 1:** Be clear and accessible for a non-technical audience.
- **Criteria 2:** Detail the audit procedures, including the audit objective and methods used.
- **Criteria 3:** Present key findings concisely, highlighting any identified issues.
- **Criteria 4:** Offer actionable recommendations for stakeholders based on audit findings.

In [ ]:
# define the report writing task for the Audit Manager
write_report = Task(
    description=dedent("""\
        Compile a comprehensive audit report using the findings from the Senior IT Auditor. This report should clearly communicate the audit's process, results, and recommendations to stakeholders.

        In summary, your report should comply with the following criteria:
        
        1. Be clear and accessible for a non-technical audience.
        2. Include a detailed overview of the audit procedures, providing context and explaining the audit objectives.
        3. Present key findings concisely, highlighting significant observations such as high-value transactions.
        4. Offer actionable recommendations for stakeholders based on the findings.
        
        Your report structure should include:

        1. **Introduction**: Brief context and purpose of the audit.
        2. **Audit Procedures**: Description of methods used and rationale.
        3. **Findings**: Summary of query results, particularly high-value transactions or anomalies.
        4. **Recommendations**: Practical suggestions addressing any issues uncovered.
        5. **Conclusion**: Summary of key insights and implications for future audits.

        The goal is to create a report that effectively communicates the audit findings and provides valuable insights to stakeholders.
        
        As the report writing expert, you are responsible for producing this report without requesting additional information. 
        Utilize the provided data and insights to complete your task effectively.
    """),
    expected_output=dedent("""\
        A full audit report presented in a clear and accessible manner.
    """),
    agent=audit_manager,  # assign the task to the audit manager agent
    max_attempts=1,
    output_file='task_03.txt'
)

With the agents now defined, we can establish the audit team. The agents will perform the audit task of Journal Entry Testing as implemented in the following section. 

## 5. Establishing the Agentic Journal Entry Testing

In this final section, we bring together all the agents we have created—the **Senior Auditor**, **Senior IT Auditor**, and **Audit Manager**—and assign them their respective tasks. Together, they form the **Audit Team**, which will collaboratively perform the full audit process.

The **Audit Team** will work sequentially:
1. The **Senior Auditor** will analyze the guidelines and extract key points.
2. The **Senior IT Auditor** will translate those points into SQL queries and run them on the payments database.
3. The **Audit Manager** will compile the findings into a comprehensive audit report.

Let’s get the team to work!

In [ ]:
# instantiate the audit team
audit_team = Crew(
    agents=[senior_auditor, senior_it_auditor, audit_manager],  # assign agents to the audit team
    tasks=[analyze_guideline, query_database, write_report],     # assign tasks to the audit team
    verbose=False,  # set logging level for output visibility
    process=Process.sequential  # set the process to sequential to ensure proper order of execution
)

Now that the **Audit Team** is instantiated, we can run the crew to perform the audit. The agents will work through their tasks sequentially, ensuring each step is completed in sequential order.

In [ ]:
# get the audit team to work
result = audit_team.kickoff();

In this lab, you successfully accomplished the following key learnings:

> 1. **Understanding Agentic Auditing:** Learned core principles of agentic auditing and how AI agents automate structured audit tasks, specifically journal entry testing.
> 2. **Implementing Agents with CrewAI:** Gained hands-on experience setting up agents in CrewAI, including a **Senior Auditor** for summarizing guidelines, a **Senior IT Auditor** for database queries, and an **Audit Manager** for report creation.
> 3. **Creating Agentic Workflows for Journal Entry Testing:** Built a sequential agent workflow to analyze guidelines, query financial records, and compile results into a complete audit report.

This lab provided insights into designing and implementing AI agents for financial auditing tasks. Through setup, task configuration, and agent collaboration, you developed skills for using agentic workflows to enhance auditing processes.
